#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [1]:
import pandas as pd
import pymongo
import date

In [11]:
client = pymongo.MongoClient("mongo")
database = client['exercices']
collection = database['kickstarter']

In [3]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [9]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [5]:
df_ks['launched'] = pd.to_datetime(df_ks['launched'], errors='coerce')
df_ks = df_ks.dropna(subset=['launched'])
df_ks.rename(columns={'ID':'_id'}, inplace=True)

l_d =df_ks.to_dict(orient='records')

### Importer les données

In [12]:
collection.insert_many(l_d)

## Question 1  

In [19]:
cur = collection.find().sort([('pledged',-1)]).limit(5)
list(cur)

[{'_id': 218218259,
  'name': 'True North ï¿½ Repurposed Sailcloth Products',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2016-06-14',
  'goal': '7500.0',
  'launched': datetime.datetime(2016, 5, 10, 15, 53, 56),
  'pledged': '9996.0',
  'state': 'successful',
  'backers': '50',
  'country': 'US',
  'usd pledged': '9996.0',
  'usd_pledged_real': 9996.0},
 {'_id': 1712352341,
  'name': 'Scott Davis - Solo Record',
  'category': 'Music',
  'main_category': 'Music',
  'currency': 'USD',
  'deadline': '2014-11-22',
  'goal': '8000.0',
  'launched': datetime.datetime(2014, 10, 23, 20, 6, 37),
  'pledged': '9995.0',
  'state': 'undefined',
  'backers': '0',
  'country': 'N,0"',
  'usd pledged': nan,
  'usd_pledged_real': 9995.0},
 {'_id': 1742823215,
  'name': 'Polco: A Digital Platform for Better Civic Communication',
  'category': 'Web',
  'main_category': 'Technology',
  'currency': 'USD',
  'deadline': '2017-08-11',
  'goal': '35000.0'

## Question 2

In [73]:
cur = collection.count_documents({"pledged" : {'$gte' : '$goal'}})
print("Number of projects that reached their goal is : ", cur)

Number of projects that reached their goal is :  65535


## Question 3

In [70]:
cur = collection.aggregate([{"$group" : {"_id" : "$category", "nomber of projets" : {"$sum" : 1}}}])
list(cur)

[{'_id': '3D Printing', 'nomber of projets': 271},
 {'_id': 'Music', 'nomber of projets': 6229},
 {'_id': 'People', 'nomber of projets': 440},
 {'_id': 'Product Design', 'nomber of projets': 8886},
 {'_id': 'Apparel', 'nomber of projets': 2827},
 {'_id': 'Apps', 'nomber of projets': 2535},
 {'_id': 'Fashion', 'nomber of projets': 3379},
 {'_id': 'Hardware', 'nomber of projets': 1431},
 {'_id': 'Literary Journals', 'nomber of projets': 118},
 {'_id': 'Webcomics', 'nomber of projets': 259},
 {'_id': 'Drama', 'nomber of projets': 871},
 {'_id': 'Pop', 'nomber of projets': 1302},
 {'_id': 'Experimental', 'nomber of projets': 357},
 {'_id': 'Young Adult', 'nomber of projets': 328},
 {'_id': 'Art', 'nomber of projets': 3358},
 {'_id': 'Events', 'nomber of projets': 322},
 {'_id': 'Classical Music', 'nomber of projets': 1064},
 {'_id': 'R&B', 'nomber of projets': 172},
 {'_id': 'Embroidery', 'nomber of projets': 49},
 {'_id': 'Faith', 'nomber of projets': 439},
 {'_id': 'Gaming Hardware', 'no

## Question 4

In [75]:
import datetime
date_iso = datetime.datetime(2016,1,1,0,0,0)
cur = collection.count_documents({"$and" : [{"country" : "FR"}, {"launched" : {"$lt" : date_iso}}]})
print("Number of French projects that were instantiated before 2016 is : ", cur)

Number of French projects that were instantiated before 2016 is :  330


## Question 5

In [51]:
cur = collection.aggregate([
    {"$match" : {"$and" : [{"country" : "US"}, {"goal" : {'$gt' : 200000}}]}}
])
list(cur)

[{'_id': 866634482,
  'name': 'A CALL TO ADVENTURE',
  'category': 'Film & Video',
  'main_category': 'Film & Video',
  'currency': 'USD',
  'deadline': '2012-09-14',
  'goal': 287000.0,
  'launched': datetime.datetime(2012, 8, 13, 23, 14, 2),
  'pledged': 1465.0,
  'state': 'failed',
  'backers': 11,
  'country': 'US',
  'usd pledged': 1465.0,
  'usd_pledged_real': 1465.0},
 {'_id': 993194166,
  'name': 'Storybricks, the storytelling online RPG',
  'category': 'Video Games',
  'main_category': 'Games',
  'currency': 'USD',
  'deadline': '2012-06-01',
  'goal': 250000.0,
  'launched': datetime.datetime(2012, 5, 1, 20, 49, 58),
  'pledged': 23680.54,
  'state': 'failed',
  'backers': 409,
  'country': 'US',
  'usd pledged': 23680.54,
  'usd_pledged_real': 23680.54},
 {'_id': 1147175344,
  'name': 'Shine On New World',
  'category': 'Theater',
  'main_category': 'Theater',
  'currency': 'USD',
  'deadline': '2013-11-04',
  'goal': 300000.0,
  'launched': datetime.datetime(2013, 9, 30, 18

## Question 6 

In [76]:
cur = collection.count_documents({"name" : {'$regex' : '.*Sport.*'}})
print( "number of name with 'Sport' in them is :", cur)


number of name with 'Sport' in them is : 323
